In [21]:
import random
import pandas as pd

def generate_student_data(num_rows):
  data = []
  student_ids = range(1, 101)  # Assuming up to 100 students
  session_ids = range(1, 501)   # Assuming up to 500 sessions
  speach = ["The Importance of Exercise", "The Impact of Technology on Education", "The Challenges and Benefits of Urbanization", "The Role of Renewable Energy in Combating Climate Change","The Impact of Globalization on Local Cultures"]
  
  for _ in range(num_rows):
    row = {
        'Student_ID': random.choice(student_ids),
        'Session_ID': random.choice(session_ids),
        'Quiz_1_Marks': random.choice([0, 20, 40, 60, 80, 100]) ,
        'Quiz_2_Marks':random.choice([0, 20, 40, 60, 80, 100]) ,
        'Quiz_3_Marks': random.choice([0, 20, 40, 60, 80, 100]) ,
        'Quiz_4_Marks': random.choice([0, 20, 40, 60, 80, 100]) ,
        'Quiz_5_Marks': random.choice([0, 20, 40, 60, 80, 100]) ,
        'speach': random.choice(speach),
        'Time_Spent_on_speach': f"{random.randint(1, 5)} min"
    }

    # Calculate Cumulative Average and Recent Quiz Trend
    quiz_scores = [row[f'Quiz_{i}_Marks'] for i in range(1, 6)]
    row['Cumulative_Average_Quiz_Score'] = round(sum(quiz_scores) / len(quiz_scores), 1)

    if quiz_scores[-1] > quiz_scores[-2]:
        row['Recent_Quiz_Trend'] = 'Improving'
    elif quiz_scores[-1] < quiz_scores[-2]:
        row['Recent_Quiz_Trend'] = 'Declining'
    else:
        row['Recent_Quiz_Trend'] = 'Stable'

    row['Score'] =  max(20, int(row['Cumulative_Average_Quiz_Score'] + random.randint(-10, 10))) 

    data.append(row)

  return pd.DataFrame(data)

# Generate and display 10 rows of new data
new_data = generate_student_data(5000)
# print(new_data)

In [22]:
new_data.to_csv('student_learning_data_speach.csv', index=False)

In [23]:
new_data

,Student_ID,Session_ID,Quiz_1_Marks,Quiz_2_Marks,Quiz_3_Marks,Quiz_4_Marks,Quiz_5_Marks,speach,Time_Spent_on_speach,Cumulative_Average_Quiz_Score,Recent_Quiz_Trend,Score
0,66,400,40,60,100,20,40,The Importance of Exercise,1 min,52.0,Improving,42
1,96,318,100,100,60,100,80,The Impact of Globalization on Local Cultures,3 min,88.0,Declining,93
2,99,263,0,0,0,100,20,The Impact of Technology on Education,3 min,24.0,Declining,24
3,79,444,20,20,80,80,20,The Challenges and Benefits of Urbanization,4 min,44.0,Declining,47
4,46,95,20,40,40,40,40,The Challenges and Benefits of Urbanization,5 min,36.0,Stable,33
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,97,311,0,100,20,100,0,The Impact of Technology on Education,5 min,44.0,Declining,38
4996,28,221,80,60,80,60,60,The Impact of Technology on Education,5 min,68.0,Stable,64
4997,71,378,80,100,0,100,100,The Role of Renewable Energy in Combating Clim...,1 min,76.0,Stable,71
4998,53,357,20,100,40,80,40,The Impact of Technology on Education,1 min,56.0,Declining,56


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder

# Load the dataset
df = pd.read_csv('student_learning_data_speach.csv')

# Selecting features and target variable
X = df[['Time_Spent_on_speach', 'Cumulative_Average_Quiz_Score','speach']]
y = df['Score']

# Preprocessing for categorical data
# Convert 'Time_Spent_Studying' to numeric (e.g., "5 hours" -> 5)
X['Time_Spent_on_speach'] = X['Time_Spent_on_speach'].str.extract('(\d+)').astype(int)

# One-Hot Encoding for categorical features
categorical_features = ['speach']
numeric_features = ['Time_Spent_on_speach', 'Cumulative_Average_Quiz_Score']

# Define Column Transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Define the model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor',  RandomForestRegressor())
])


# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Mean Absolute Error: {mae}")
print(f"Root Mean Squared Error: {rmse}")

C:\Users\user\AppData\Local\Temp\ipykernel_20284\2279156920.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Time_Spent_on_speach'] = X['Time_Spent_on_speach'].str.extract('(\d+)').astype(int)


Mean Absolute Error: 5.205932027420815
Root Mean Squared Error: 6.216685027769659


In [2]:
# Predicting Exam_Score for new data
new_data = pd.DataFrame({
    'Time_Spent_on_speach': ['4 min'],
    'Cumulative_Average_Quiz_Score': [60],
    'speach':['The Importance of Exercise']
})

# Convert 'Time_Spent_Studying' to numeric
new_data['Time_Spent_on_speach'] = new_data['Time_Spent_on_speach'].str.extract('(\d+)').astype(int)

predicted_scores = model.predict(new_data)
print(f"Predicted Scores: {predicted_scores}")

Predicted Scores: [61.51775608]


In [20]:
import joblib

# Save the model to a file
joblib.dump(model, 'Mark_predictor.pkl')


['Mark_predictor.pkl']

In [1]:
pip list

Package                       Version
----------------------------- ------------------
absl-py                       1.4.0
aio-pika                      8.2.3
aiofiles                      23.2.1
aiogram                       2.25.2
aiohttp                       3.8.6
aiohttp-cors                  0.7.0
aiohttp-retry                 2.8.3
aioice                        0.9.0
aiormq                        6.4.2
aiortc                        1.7.0
aiosignal                     1.3.1
annotated-types               0.5.0
anyio                         3.7.1
api.video                     1.3.0
APScheduler                   3.9.1.post1
argon2-cffi                   23.1.0
argon2-cffi-bindings          21.2.0
arrow                         1.3.0
asttokens                     2.4.0
astunparse                    1.6.3
async-lru                     2.0.4
async-timeout                 4.0.3
attrs                         23.2.0
av                            11.0.0
Babel                         2.14.0
